## Convert filter_blast function to use pandas

In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys
sys.path.append('/Users/hcross/analysis/git_projects/genbanking/gb_parsers/')
from blast_parsers import blast_parser

In [3]:
def filter_blast(blastDF, blast_parameters):
    """Filter a blast result dictionary based on variable parameters using pandas"""
    ## >> have to add more to following
    field_match = {'SUB_LENGTH':'slen','MIN_QSTART':'qstart','MAX_QSTART':'qstart','MIN_QEND':'qend','MAX_QEND':'qend','MIN_SEND':'send','MAX_SEND':'send','MIN_BITSCORE':'bitscore','MIN_POSITIVE':'positive','MAX_GAPOPEN':'gapopen','MAX_GAPS':'gaps','IN_TITLE':'stitle','IN_SSEQ':'sseq','MIN_SSTART':'sstart','MAX_SSTART':'sstart','MIN_QCOV':'qcovs','MIN_LENGTH':'length','MIN_PCT_ID':'pident','MAX_MISMATCH':'mismatch','MAX_EVALUE':'evalue','IN_SUBJECT_ID':'sseqid'}
    mins = ['MIN_LENGTH','MIN_PCT_ID','MIN_SSTART','MIN_QCOV','SUB_LENGTH','MIN_QSTART','MIN_QEND','MIN_SEND','MIN_BITSCORE','MIN_POSITIVE'] 
    maxes = ['MAX_MISMATCH','MAX_EVALUE','MAX_SSTART','MAX_QSTART','MAX_QEND','MAX_SEND','MAX_GAPOPEN','MAX_GAPS'] 
    containers = ['IN_SUBJECT_ID','IN_TITLE','IN_SSEQ']
    filtered_blast = {}
    blast_parameters.pop('FORMAT')
    num_conditions = len(blast_parameters)
    # for now have to build it as a string and then run 
    string_list = []
    for k,v in blast_parameters.iteritems():
        # blast2 = blast_table.loc[blast_table['pident'] > 35]
        blast_field = field_match[k]
        string = "blastDF[blast_field]"
        if k in mins:
            #string = string + "> "+v+"]"
            string = string + "> %s" %(v)
            #if hit_dict[blast_field] > v:   
        elif k in maxes:
            string = string + "< %s" %(v)
            #if hit_dict[blast_field] < v:
                
        elif k in containers:
            # bb5 = bb[bb['stitle'].str.contains('Maireana')]
            #string = string+".str.contains("+v+")]"
            string = string+".str.contains('%s')" % (v)
            #if v in hit_dict[blast_field]: # need to fix this, was in v
        string_list.append(string)
    if num_conditions == 1:
        cmd_string = string_list[0]
        final_string = "filt_blastDF = blastDF["+cmd_string+"]"
        exec(final_string)
        
    # for multiple conditions:
    #blast3 = blast_table[(blast_table['pident'] > 35) & (blast_table['length'] > 100)]
    else:
        initial = "filt_blastDF = blastDF["
        for cmd in string_list:
            new_string = "("+cmd+")"
            initial = initial+new_string+"&"
        semifinal = initial[:-1]
        final = semifinal+"]"
        exec(final)
    return filt_blastDF
    

### Test with big file

In [7]:
test1_names = ['qseqid','sseqid','stitle','extra','evalue','bitscore','qcovs','length','pident','nident','qstart','qend','sstart','send','qseq','sseq']

In [8]:
test = pd.read_table('testblast1.txt', sep='\t', names=test1_names)

In [9]:
test.head()

,qseqid,sseqid,stitle,extra,evalue,bitscore,qcovs,length,pident,nident,qstart,qend,sstart,send,qseq,sseq
0,cam03.cf15_322,gi|564614030|gb|KF669512.1|,Acremonium sp. 904C internal transcribed space...,partial sequence; 5.8S ribosomal RNA gene and...,5.580000e-30,139,100,78,98.718,77,1,78,473,550,GCGGTGCGGCCAAGCCGTAAAACACCCCACTTCTTCAAGGTTGACC...,GCGGTGCGGCCACGCCGTAAAACACCCCACTTCTTCAAGGTTGACC...
1,cam03.cf15_322,gi|312434386|gb|HQ607908.1|,Ascomycota sp. AR-2010 isolate ATT243 18S ribo...,"l RNA gene, partial sequence; internal transcr...",9.340000e-28,132,100,78,97.436,76,1,78,505,581,GCGGTGCGGCCAAGCCGTAAAACACCCCACTTCTTCAAGGTTGACC...,GCGGTGCGGCCACGCCGTAAAACACCCCACTTC-TCAAGGTTGACC...
2,cam03.cf15_322,gi|256855623|emb|FN397253.1|,Uncultured fungus genomic DNA sequence contain...,"18S rRNA gene, ITS1, 5.8S rRNA gene, ITS2 and ...",9.340000e-28,132,100,78,97.436,76,1,78,518,594,GCGGTGCGGCCAAGCCGTAAAACACCCCACTTCTTCAAGGTTGACC...,GCGGTGCGGCCACGCCGTAAAACACCCCACTTC-TCAAGGTTGACC...
3,cam03.cf15_322,gi|1030027947|gb|KU931422.1|,Uncultured fungus clone WJ139 18S ribosomal RN...,"ne, partial sequence",3.360000e-27,130,100,80,96.250,77,1,78,535,614,GCGGTGCGGCCAAGCCGTAAAACACCCCACTTCTTC--AAGGTTGA...,GCGGTGCGGCCACGCCGTAAAACACCCCACTTCTTCCAAAGGTTGA...
4,cam03.cf15_322,gi|1005260240|emb|HG937108.1|,Uncultured Acremonium genomic DNA containing 1...,"RNA gene, ITS1, 5.8S rRNA gene, ITS2 and 28S r...",3.360000e-27,130,94,73,98.630,72,6,78,520,592,GCGGCCAAGCCGTAAAACACCCCACTTCTTCAAGGTTGACCTCGGA...,GCGGCCACGCCGTAAAACACCCCACTTCTTCAAGGTTGACCTCGGA...


In [10]:
test2 = test[test['stitle'].str.contains('Acremonium')]

In [12]:
test2

,qseqid,sseqid,stitle,extra,evalue,bitscore,qcovs,length,pident,nident,qstart,qend,sstart,send,qseq,sseq
0,cam03.cf15_322,gi|564614030|gb|KF669512.1|,Acremonium sp. 904C internal transcribed space...,partial sequence; 5.8S ribosomal RNA gene and...,5.580000e-30,139,100,78,98.718,77,1,78,473,550,GCGGTGCGGCCAAGCCGTAAAACACCCCACTTCTTCAAGGTTGACC...,GCGGTGCGGCCACGCCGTAAAACACCCCACTTCTTCAAGGTTGACC...
4,cam03.cf15_322,gi|1005260240|emb|HG937108.1|,Uncultured Acremonium genomic DNA containing 1...,"RNA gene, ITS1, 5.8S rRNA gene, ITS2 and 28S r...",3.360000e-27,130,94,73,98.630,72,6,78,520,592,GCGGCCAAGCCGTAAAACACCCCACTTCTTCAAGGTTGACCTCGGA...,GCGGCCACGCCGTAAAACACCCCACTTCTTCAAGGTTGACCTCGGA...
8,cam03.cf15_322,gi|1373300|gb|U57672.1|ACU57672,Acremonium chrysogenum rDNA internal transcrib...,pacers 1 and 2 and 5.8S rRNA gene,3.360000e-27,130,94,73,98.630,72,6,78,480,552,GCGGCCAAGCCGTAAAACACCCCACTTCTTCAAGGTTGACCTCGGA...,GCGGCCACGCCGTAAAACACCCCACTTCTTCAAGGTTGACCTCGGA...


In [13]:
test2 = test[test['extra'].str.contains('rRNA')]

In [17]:
test2_names = ['qseqid','sseqid','taxon', 'stitle','extra','evalue','bitscore','qcovs','length','pident','nident','qstart','qend','sstart','send','qseq','sseq']

In [18]:
testDf2 = pd.read_table('testblast2.txt', sep='\t', names=test2_names)

In [19]:
test3 = testDf2[testDf2['taxon'].str.contains('Acremonium')]

In [20]:
test3 = testDf2[testDf2['stitle'].str.contains('rRNA')]

In [21]:
test4 = testDf2[testDf2['sseq'].str.contains('TTCTT')]

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [22]:
test3_names = ['qseqid','sseqid','taxon', 'stitle','extra','evalue','bitscore','qcovs','length','pident','nident','qstart','qend','sstart','send','qseq','sseq1','sseq2']

In [28]:
testDf3 = pd.read_table('testblast3.txt', sep='\t', names=test3_names)

In [29]:
test5 = testDf3[testDf3['sseq1'].str.contains('TTCTT')]

In [37]:
test6 = testDf3[testDf3['sseq2'].str.contains('TTCTT')]

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [34]:
test7 = testDf3.sseq2.apply(str)

In [35]:
test6 = test7[test7['sseq2'].str.contains('TTCTT')]

KeyError: 'sseq2'

In [36]:
test7

0    nan
1    nan
2    nan
3    nan
4    nan
5    nan
6    nan
7    nan
8    nan
9    nan
Name: sseq2, dtype: object